<div class="alert alert-block alert-info">
Author:<br>Felix Gonzalez, P.E. <br>
</div>

This notebook shows how to obtain input from the USB Camera. The input of the USB Camera will be used to allow image detection within the video feed.

# Table of Contents
[](#)

[](#)

[](#)

[](#)

[](#)

[](#)

# Python Libraries

In [1]:
import os
# HW Transformers to 0 fixes issue with USB Webcam Loading very slow in OpenCV
os.environ["OPENCV_VIDEOIO_MSMF_ENABLE_HW_TRANSFORMS"] = "0"

In [2]:
import cv2 as cv
import ipywidgets as widgets
import threading
from IPython.display import display, Image

# Working with Video Inputs
[Return to Table of Contents](#Table-of-Contents)

There are varios ways to define the video camera instance and obtain a video feed. In this notebook two ways are shown:
1. Camera instance within the Jupyter Notebook
2. Camera instance in a new window.

We will mainly use OpenCV Python Library which is a library that allows you to work opening video instances. OpenCV also has various capabilities including detecing facial bounding boxes which we will see below.

### Camera Instance: in Notebook

Running Open CV in Notebook with Stop Button.

References: 
- https://medium.com/@abauville/display-your-live-webcam-feed-in-a-jupyter-notebook-using-opencv-d01eb75921d1

In [3]:
# WebCam input with Stop button
stopButton = widgets.ToggleButton(value = False,
                                  description = 'Stop',
                                  disabled = False,
                                  button_style= 'danger', # 'success', 'info', 'warning', 'danger' or ''
                                  tooltip = 'Description',
                                  icon = 'square' # (FontAwesome names without the `fa-` prefix)
                                 )

# Display function
def view(button):
    capture = cv.VideoCapture(1) # Video capture object. Camera 0 is the integrated camera while 1 is USB Camera.
    # Frame Width/Height can be commented out. Common resolutions: 1920x1080, 1600x900, 1280x800, 640x480.
    capture.set(cv.CAP_PROP_FRAME_WIDTH, 1280) 
    capture.set(cv.CAP_PROP_FRAME_HEIGHT, 800)
    
    display_handle = display(None, display_id=True)

    #while True: # Both while statements seem to work.
    while(capture.isOpened()):
        _, frame = capture.read() # Reads and captures the image frame
        frame = cv.flip(frame, 1) # Reverses image if webcam reverses
        _, frame = cv.imencode('.jpeg', frame) # Reads frame from buffer in memory
        display_handle.update(Image(data=frame.tobytes())) # Uses the Jupyter Notebook Display to show in Notebook.
        if stopButton.value == True:
            capture.release()
            cv.destroyAllWindows()
            display_handle.update(None)

In [4]:
# Run the view function. Note takes about 4 minutes to load video.
display(stopButton)
thread = threading.Thread(target = view, 
                          args = (stopButton,))
thread.start()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None

### Camera Instance: in Notebook with Facial Bounding Box

In [5]:
# WebCam input with Stop button
stopButton = widgets.ToggleButton(value = False,
                                  description = 'Stop',
                                  disabled = False,
                                  button_style= 'danger', # 'success', 'info', 'warning', 'danger' or ''
                                  tooltip = 'Description',
                                  icon = 'square' # (FontAwesome names without the `fa-` prefix)
                                 )

# Display function
def view_wfacebox(button):
    capture = cv.VideoCapture(1) # Video capture object. Camera 0 is the integrated camera while 1 is USB Camera.

    # Load the cascade
    face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
    # Frame Width/Height can be commented out. Common resolutions: 1920x1080, 1600x900, 1280x800, 640x480.
    capture.set(cv.CAP_PROP_FRAME_WIDTH, 640)
    capture.set(cv.CAP_PROP_FRAME_HEIGHT, 480)
    
    display_handle = display(None, display_id=True)

    #while True: # Both while statements seem to work.
    while(capture.isOpened()):
        _, frame = capture.read() # Reads and captures the image frame
        frame = cv.flip(frame, 1) # Reverses image if webcam reverses
        
        # Convert to grayscale
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        # Detect the faces
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        # Draw the rectangle around each face
        for (x, y, w, h) in faces:
            cv.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        _, frame = cv.imencode('.jpeg', frame) # Reads frame from buffer in memory
        display_handle.update(Image(data=frame.tobytes())) # Uses the Jupyter Notebook Display to show in Notebook.
        if stopButton.value == True:
            capture.release()
            cv.destroyAllWindows()
            display_handle.update(None)

In [6]:
# Run the view function. Note takes about 4 minutes to load video.
display(stopButton)
thread = threading.Thread(target = view_wfacebox, 
                          args = (stopButton,))
thread.start()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None

### Camera Instance: New Window

In [7]:
# Note takes about 5 minutes to load image in new window.

capture = cv.VideoCapture(1) # Video capture object. Camera 0 is the integrated camera while 1 is USB Camera.
# Frame Width/Height can be commented out. Common resolutions: 1920x1080, 1600x900, 1280x800, 640x480.
capture.set(cv.CAP_PROP_FRAME_WIDTH, 1280) 
capture.set(cv.CAP_PROP_FRAME_HEIGHT, 800)

while(capture.isOpened()):
    _, frame = capture.read() # Reads and captures the image frame
    
    cv.imshow('Camera Video', frame) # Display in new window
    
    key = cv.waitKey(30) & 0xff # Reads keys every defined milliseconds to enter break below on key press
    if key & 0xFF == ord('q') or key == 27 or 'x' == chr(key & 255): # To exit, press q, x, or ESC.
        break
        
# Release the VideoCapture object
capture.release()
cv.destroyAllWindows()

### Camera Instance: New Window with Facial Bounding Box

In [8]:
capture = cv.VideoCapture(1) # Video capture object. Camera 0 is the integrated camera while 1 is USB Camera.

# Load the cascade
face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Frame Width/Height can be commented out. Common resolutions: 1920x1080, 1600x900, 1280x800, 640x480.
capture.set(cv.CAP_PROP_FRAME_WIDTH, 640)
capture.set(cv.CAP_PROP_FRAME_HEIGHT, 480)

while(capture.isOpened()):
    # Read the frame
    _, frame = capture.read()

    # Convert to grayscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Detect the faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Draw the rectangle around each face
    for (x, y, w, h) in faces:
        cv.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Display
    cv.imshow('Camera Video', frame)

    key = cv.waitKey(30) & 0xff # Reads keys every defined milliseconds to enter break below on key press
    if key & 0xFF == ord('q') or key == 27 or 'x' == chr(key & 255): # To exit, press q, x, or ESC.
        break
        
# Release the VideoCapture object
capture.release()
cv.destroyAllWindows()

# NOTEBOOK END